In [2]:
import plotly
import pandas as pd
plotly.tools.set_credentials_file(username='cclipp', api_key='PXEF8jIWI8E52Cst97E4')

In [3]:
df = pd.read_csv(r'D:\uci_data\donorschoose-application-screening\train\train.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182080 entries, 0 to 182079
Data columns (total 16 columns):
id                                              182080 non-null object
teacher_id                                      182080 non-null object
teacher_prefix                                  182076 non-null object
school_state                                    182080 non-null object
project_submitted_datetime                      182080 non-null object
project_grade_category                          182080 non-null object
project_subject_categories                      182080 non-null object
project_subject_subcategories                   182080 non-null object
project_title                                   182080 non-null object
project_essay_1                                 182080 non-null object
project_essay_2                                 182080 non-null object
project_essay_3                                 6374 non-null object
project_essay_4                        

In [6]:
subdf = df[['school_state', 'project_grade_category', 'project_is_approved','teacher_number_of_previously_posted_projects']]
subdf.columns = ['state', 'category', 'approve', 'projects']


In [7]:
subdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182080 entries, 0 to 182079
Data columns (total 4 columns):
state       182080 non-null object
category    182080 non-null object
approve     182080 non-null int64
projects    182080 non-null int64
dtypes: int64(2), object(2)
memory usage: 5.6+ MB


In [8]:
subdf.describe()

,approve,projects
count,182080.000000,182080.000000
mean,0.847682,11.237055
std,0.359330,28.016086
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,2.000000
75%,1.000000,9.000000
max,1.000000,451.000000


In [9]:
import re
from nltk.corpus import stopwords

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()# lowercase text  
    text = REPLACE_BY_SPACE_RE.sub(' ',text)# replace REPLACE_BY_SPACE_RE symbols by space in text    
    text = BAD_SYMBOLS_RE.sub('',text)# delete symbols which are in BAD_SYMBOLS_RE from text    
    temp = [s.strip() for s in text.split() if s not in STOPWORDS]# delete stopwords from text
    new_text = ''
    for i in temp:
        new_text +=i+' '
    text = new_text
    return text.strip()

temp_data = df.dropna(subset=['project_resource_summary'])
# converting into lowercase
temp_data['project_resource_summary'] = temp_data['project_resource_summary'].apply(lambda x: " ".join(x.lower() for x in x.split()))
temp_data['project_resource_summary'] = temp_data['project_resource_summary'].map(text_prepare)


from wordcloud import WordCloud

wordcloud = WordCloud(max_font_size=50, width=600, height=300).generate(' '.join(temp_data['project_resource_summary'].values))
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.title("Top resources needed for the project", fontsize=35)
plt.axis("off")
plt.show() 

ModuleNotFoundError: No module named 'wordcloud'